# LDA and Top-30 words visualization

In this exercise, I used my Twitter archive in Russian (about 22.000 tweets) as corpus. LDA topic modelling is made with Gensim library. 

In [16]:
import nltk
import string
import pandas as pd
from gensim.corpora import Dictionary
from gensim import corpora
import pickle
import gensim

#define all words and signs that are to be removed from the text
stopwords = nltk.corpus.stopwords.words('russian')
stopwords_delete = ['никогда', 'нельзя', 'всегда', "между", "сейчас", "хорошо", "перед"]
stopwords_add = ['это', 'который', "хотя", "кстати", "обо", "ещё", "меж", "зато", "пусть", "ага", "этот", "это", "почему", "видимо", "кажется",
                 "весь", "ты", "он", "она", "оно", "мы", "вы", "кто", "что", "сам", "сама", "само", "свой", "наш", "ваш", "их", "тот", "та", "те", 
                 "то", "раз", "твой", "мой", "кой", "кое", "все", "весь", "всё", "быть", "тот", "кроме", "причем", "наверное", "около",
                 "таки", "такой", "какой", "каждый", "который", "вместо", "например", "вообще", "значит", "б", "д", 
                 "е", "ж", "з", "к", "л", "м", "н", "п", "р", "с", "ф", "ч", "ц", "ш", "щ", "ь", "ъ","э", "ю", "i", "the", "on", "a", "are"
                 "is", "was", "were", "to", "as", "so", "am", "about", "for", "re", "you", "we", "they", "us", "them", "me", "though", "although", 
                 "all", "or", "and", "some", "until", "an", "will", "no", "yes", "not", "with", "your", "this"]

new_stopwords = []
for word in stopwords:
	if word not in stopwords_delete:
		new_stopwords.append(word)
stopwords = new_stopwords
if len(stopwords_add) != 0:
	stopwords += stopwords_add
punctuation = list(string.punctuation)
punctuation += ['–', '—', '"', "¿", "¡", "``", "''", "..."]
stop = stopwords + punctuation     

For LDA, we just need all texts to be tokenized. 

In [17]:
def prepare_text_for_lda(text):
    tokens = text.split()
    tokens = [token for token in tokens if len(token) > 4] #we use only "long" words, they are usually more significant
    tokens = [token for token in tokens if token not in stop] #filter stopwords and punctuation
    return tokens

In [18]:
#We load CSV table with the archive of tweets. Previuosly, I cleaned the text from unsernames, smileys and hashtags, 
#and lemmatized it to reduce the number of tokens. You can see how to do this for Russian here: https://github.com/ZotovaElena/Tweets-Preprocessing
tweets = pd.read_csv('tweets_clean.csv', sep='\t')
tweets = tweets.fillna('')
tweets_text = tweets.text_lemmatized.values
tweets_text = list(tweets.text_lemmatized.values)

#create a list of tweets tokenized
text_data = []
for line in tweets_text: 
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)
            
#create a dictionary from the data, then convert to bag-of-words corpus and save the dictionary and corpus for future use
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

#Ask LDA to find some topics in the data
NUM_TOPICS = 15
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model15.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.037*"песня" + 0.030*"находить" + 0.030*"барнаул" + 0.029*"место" + 0.026*"рядом" + 0.022*"показывать" + 0.020*"несколько" + 0.019*"машина" + 0.016*"центр" + 0.015*"радио"')
(1, '0.039*"русский" + 0.031*"нравиться" + 0.029*"видеть" + 0.029*"испанский" + 0.029*"москва" + 0.027*"слово" + 0.027*"очень" + 0.020*"ездить" + 0.020*"дорога" + 0.019*"становиться"')
(2, '0.046*"читать" + 0.031*"оказываться" + 0.027*"обычный" + 0.024*"книга" + 0.022*"мужчина" + 0.020*"слышать" + 0.017*"завтра" + 0.017*"ребенок" + 0.014*"заниматься" + 0.014*"отлично"')
(3, '0.052*"телефон" + 0.041*"купить" + 0.037*"смотреть" + 0.035*"магазин" + 0.035*"новый" + 0.034*"деньги" + 0.027*"фильм" + 0.019*"покупать" + 0.018*"страна" + 0.016*"довольно"')
(4, '0.085*"хорошо" + 0.062*"работа" + 0.038*"ходить" + 0.038*"ехать" + 0.021*"собака" + 0.018*"карта" + 0.016*"нужно" + 0.015*"проходить" + 0.015*"интересно" + 0.015*"белый"')
(5, '0.098*"хороший" + 0.028*"сколько" + 0.028*"правда" + 0.023*"испания" + 0.023*"погода

It is not very easy to make LDA topics in Gensim human readable. Here is a nice way to visualize them. 

In [19]:
import pyLDAvis.gensim
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary) 

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.113599  0.046553       1        1  8.517476
3     -0.067937  0.173836       2        1  7.676863
5     -0.110868 -0.043704       3        1  7.377845
4      0.142437 -0.217048       4        1  7.350982
14     0.168874  0.137598       5        1  6.732405
0      0.027861 -0.165413       6        1  6.645657
9     -0.059376  0.058397       7        1  6.537835
6      0.094567  0.059600       8        1  6.465833
10     0.051845  0.016615       9        1  6.393260
12    -0.166793 -0.072316      10        1  6.368255
11     0.001453  0.115700      11        1  6.267982
7     -0.129347 -0.089816      12        1  6.258974
13     0.160460 -0.031986      13        1  6.029257
8      0.046397  0.036222      14        1  5.755906
2     -0.045976 -0.024238      15        1  5.621463, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
380   Default  737.000000        сегодня  737.000000  30.0000  30.0000
63    Default  723.000000          знать  723.000000  29.0000  29.0000
18    Default  745.000000        хороший  745.000000  28.0000  28.0000
5     Default  644.000000         хорошо  644.000000  27.0000  27.0000
7     Default  467.000000         работа  467.000000  26.0000  26.0000
681   Default  434.000000         писать  434.000000  25.0000  25.0000
234   Default  407.000000         думать  407.000000  24.0000  24.0000
479   Default  392.000000        спасибо  392.000000  23.0000  23.0000
547   Default  385.000000         просто  385.000000  22.0000  22.0000
125   Default  385.000000       работать  385.000000  21.0000  21.0000
709   Default  413.000000        телефон  413.000000  20.0000  20.0000
15    Default  335.000000         делать  335.000000  19.0000  19.0000
83    Default  325.000000        сказать  325.000000  18.0000  18.0000
70    Default  320.000000         первый  320.000000  17.0000  17.0000
812   Default  322.000000        твиттер  322.000000  16.0000  16.0000
626   Default  310.000000       понимать  310.000000  15.0000  15.0000
747   Default  341.000000        русский  341.000000  14.0000  14.0000
417   Default  390.000000         купить  390.000000  13.0000  13.0000
181   Default  292.000000          город  292.000000  12.0000  12.0000
23    Default  285.000000          пойти  285.000000  11.0000  11.0000
441   Default  290.000000        сделать  290.000000  10.0000  10.0000
146   Default  267.000000         читать  267.000000   9.0000   9.0000
105   Default  362.000000         ходить  362.000000   8.0000   8.0000
693   Default  289.000000          ехать  289.000000   7.0000   7.0000
708   Default  292.000000       смотреть  292.000000   6.0000   6.0000
212   Default  262.000000     посмотреть  262.000000   5.0000   5.0000
140   Default  281.000000        магазин  281.000000   4.0000   4.0000
391   Default  247.000000       написать  247.000000   3.0000   3.0000
453   Default  335.000000          новый  335.000000   2.0000   2.0000
55    Default  251.000000          никто  251.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
1916  Topic15   97.006004         завтра   98.207642   2.8663  -4.0880
1538  Topic15   81.191101        отлично   82.392738   2.8639  -4.2659
507   Topic15   74.827415   представлять   76.029053   2.8626  -4.3476
489   Topic15   71.843422        возраст   73.045059   2.8620  -4.3883
750   Topic15   71.103111        молодой   72.304749   2.8618  -4.3986
2027  Topic15   70.821114         ужасно   72.022751   2.8618  -4.4026
2355  Topic15   69.343430   использовать   70.545067   2.8614  -4.4237
858   Topic15   65.745377       возможно   66.947014   2.8605  -4.4770
1203  Topic15   62.776661       основной   63.978302   2.8596  -4.5232
1100  Topic15   62.709732        детский   63.911373   2.8596  -4.5242
156